# JuliaによるtidyなData Handling

----
## はじめに  
プログラミング言語Rで超有名なパッケージ`tidyr`&`dplyr`と同様の処理を行う関数がJuliaにもあるので，それをまとめる。

パッケージのマニュアル等は[こちら](https://juliadata.github.io/DataFrames.jl/stable/)

----
## DataFrameとは

juliaやR， pythonには行列や配列とは異なる形式としてDataFrame (データフレーム) と呼ばれるデータ形式が存在する。データフレームは名前付き行列や配列と似たような形である。配列と異なる点は，明示的に型を変換しなくても，複数の異なる型の列を保持できる点である。

もちろんUnionやAnyなどの型を利用すれば，複数の異なる型の要素を維持した配列を作ることはできるが，列ごとに意味を持ったまとまりをもつような表現形式（たとえば観察データを集計したデータテーブル）では列ごとに型を管理できるデータフレームのほうが都合が良い。

In [1]:
using DataFrames, Random;

In [20]:
df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"], C = randn(4))

,A,B,C
,Int64,String,Float64
1,1,M,0.445517
2,2,F,-0.630496
3,3,F,1.21081
4,4,M,1.11362


In [7]:
mat = [1 "M"; 2 "M"]

2×2 Array{Any,2}:
 1  "M"
 2  "M"

----
## 基本的な操作  

### 列名の取り出し

データフレームにアクセスするときは`Symbols`を使い，`df[:col]`, `df[:, :A]`, `df.A` という3つの方法で列の要素を取り出すことができる。

In [21]:
df[:A] == df[:, :A] == df.A

true

----
## tidyな操作


## pipe operator
tidyな操作で不可欠になるのが，データフレームとパイプ演算子です。

パイプ演算子とは，複数の関数を連続して適用するような一連の流れを，可読性を維持したまま実行するための演算子です。直感的には関数を，まるで足し算でもするような感覚で使うことに近いです。

パイプを使うメリットは可読性の高さだけではありません。Workspace上の変数の節約にも使えます。

juliaにけるパイプ`|>`はBaseのoperators.jlにコードが存在しています。Rだと`magrittr`と呼ばれるtidyverse系のパッケージをインストールする必要があるのに対して，juliaでは起動してすぐ使えます。
https://github.com/JuliaLang/julia/blob/master/base/operators.jl

驚くべきことにコードはたったの一行です。
```
|>(x, f) = f(x)
```

Rだと，31行くらいです。おそろしくjuliaはシンプルに書かれています。

```
function (lhs, rhs) 
{
    parent <- parent.frame()
    env <- new.env(parent = parent)
    chain_parts <- split_chain(match.call(), env = env)
    pipes <- chain_parts[["pipes"]]
    rhss <- chain_parts[["rhss"]]
    lhs <- chain_parts[["lhs"]]
    env[["_function_list"]] <- lapply(1:length(rhss), function(i) wrap_function(rhss[[i]], 
        pipes[[i]], parent))
    env[["_fseq"]] <- `class<-`(eval(quote(function(value) freduce(value, 
        `_function_list`)), env, env), c("fseq", "function"))
    env[["freduce"]] <- freduce
    if (is_placeholder(lhs)) {
        env[["_fseq"]]
    }
    else {
        env[["_lhs"]] <- eval(lhs, parent, parent)
        result <- withVisible(eval(quote(`_fseq`(`_lhs`)), env, 
            env))
        if (is_compound_pipe(pipes[[1L]])) {
            eval(call("<-", lhs, result[["value"]]), parent, 
                parent)
        }
        else {
            if (result[["visible"]]) 
                result[["value"]]
            else invisible(result[["value"]])
        }
    }
}
```

欠点もあって，残念ながらjuliaのパイプ演算の機能はRよりやや書く手間が多いです。  
しかし，多くの場面では快適に，問題なく使えます。

In [ ]:
using RDatasets
iris = RDatasets.dataset("datasets", "iris")